In [1]:

import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import matplotlib.dates
from datetime import datetime

import plotly.express as px
from difflib import SequenceMatcher

In [2]:
path_1 = 'data/src/Reporte_Reclamos 2018.xls'
path_2 = 'data/src/Reporte_Reclamos 2019.xls'
path_3 = 'data/src/Reporte_Reclamos 2020.xlsx'
path_4 = 'data/src/Reporte_Reclamos 2021.xlsx'

paths = [path_1, path_2, path_3, path_4]

data_reclamos = pd.DataFrame()
for p in paths:
    data_temp = pd.read_excel(p, index_col=0, header=0)
    data_reclamos = pd.concat([data_reclamos,data_temp])

In [3]:
data_reclamos.head()

,Fase1_presen,TEMPORAL,CALIFFINAL,Fase4_generac,Fase5_revisac,Fase6_enviac,Fase7_firma_ACUSE,N° Reclamo,Nombre,N° Docum.,...,Nombre Usuario,Descripción Fase,Canal Ingreso,Medio Respuesta,Reclamo/requerim,Hace carta,Correo,Cta. Ahorro,Descripcion,Motivo.1
N°,,,,,,,,,,,,,,,,,,,,,
1,31/12/2018,17/01/2019,17/01/2019,21/01/2019,21/01/2019,17/01/2019,17/01/2019,002718-7736,ADAN HOLGUIN SOLIS,42672067,...,ANGLAS/RAMIREZ/ELSA/MARIELA,Firma,PÁGINA WEB BN,Correo Electrónico,RECLAMO,GGMA,adansmith1984@hotmail.com,4052857215.0,"EN FECHA 20 DE DICIEMBRE, EN HORAS DE LA TARDE...",NaN
2,31/12/2018,03/01/2019,03/01/2019,03/01/2019,03/01/2019,03/01/2019,03/01/2019,067618-34,ADANAQUE TIZON MARIA LILIANA,45056104,...,MARCHAN/GARCIA/JORGE,Firma,AGENCIA BN,Carta en su domicilio,RECLAMO,TCML,NaN,4676106532.0,RETIRO Y NO DIO EFECTIVO NI BOUCHER,NaN
3,31/12/2018,16/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,002718-7726,ADEMIR CHAVEZ SANCHEZ,41311452,...,WONG/MOTTA/ALEXANDRA,Firma,PÁGINA WEB BN,Carta en su domicilio,RECLAMO,GBNL,CONTACTAME.OK@GMAIL.COM,NaN,EL DIA 29 DE DICIEMBRE A LAS 18:00 EFECTUE UN ...,NaN
4,31/12/2018,09/01/2019,09/01/2019,09/01/2019,09/01/2019,09/01/2019,09/01/2019,315118-10281,AGUERO OLIVAS JOHN RICHARD,40383052,...,VALENCIA/MEDINA/TANIA IBET,Firma,MESA DE AYUDA (TELEFÓNO),Correo Electrónico,RECLAMO,RRBM,richaragol@gmail.com,4044548211.0,TITULAR MANIFIESTA QUE SE ACERCO A CAJERO DEL ...,NaN
5,31/12/2018,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,034618-9,AGUILAR VEGA JULIO EDUARDO,45451586,...,HERRERA/MEDINA/NESTOR,Firma,AGENCIA BN,Correo Electrónico,RECLAMO,SJAR,aguilar_88_2013@hotmail.com,4346109217.0,SOLICITA DEVOLUCION DE DINERO QUE NO FUE DISPE...,NaN


In [4]:
# data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].fillna(0)
# data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].astype(int)

### Correccion de nombre de ciudades

Se incorporo un archivo de centros poblados para facilitarnos la correcion

In [5]:
path_geopolitico = 'data/src/BaseCentrosPoblados.csv'
data_geopolitico = pd.read_csv(path_geopolitico, delimiter=';')
data_geopolitico = data_geopolitico.groupby(['DEPARTAMEN','PROVINCIA','DISTRITO']).agg({ 'POB': "sum"})
data_geopolitico = data_geopolitico.sort_values(by=['DEPARTAMEN','PROVINCIA','DISTRITO']).reset_index()

# correccion de departamentos
def similars(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return resp
    if max_val >= 0.8:
        return resp
    return text

def similars2(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return max_val
    return max_val

lista = list(np.unique(data_geopolitico['DEPARTAMEN']))
data_reclamos['DEPARTAMEN'] = data_reclamos['Departam. Domic.'].apply(similars, args=(lista,))
#data_reclamos['DEPARTAMEN_VAL'] = data_reclamos['Departam. Domic.'].apply(similars2, args=(lista,))

lista = list(np.unique(data_geopolitico[data_geopolitico['DEPARTAMEN']=='LIMA']['PROVINCIA']))
data_reclamos['PROVINCIA'] = data_reclamos.apply(lambda x: similars(x['Provincia Domic.'], lista) if x['Departam. Domic.']=='LIMA' else x['Provincia Domic.'],axis=1)
#data_reclamos['PROVINCIA_VAL'] = data_reclamos.apply(lambda x: similars2(x['Provincia Domic.'], lista) if x['Departam. Domic.']=='LIMA' else 0,axis=1)

lista = list(np.unique(data_geopolitico[(data_geopolitico['DEPARTAMEN']=='LIMA') & (data_geopolitico['PROVINCIA']=='LIMA')]['DISTRITO']))
data_reclamos['DISTRITO'] = data_reclamos.apply(lambda x: similars(x['Distrito Domic.'], lista)  if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else x['Distrito Domic.'],axis=1)
data_reclamos['DISTRITO_VAL'] = data_reclamos.apply(lambda x: similars2(x['Distrito Domic.'], lista) if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else 0,axis=1)

data_reclamos['IS_LIMA'] = data_reclamos.apply(lambda x: True if x['DISTRITO_VAL']>=0.8 else False, axis=1)

del data_reclamos['DISTRITO_VAL']
del data_reclamos['Departam. Domic.']
del data_reclamos['Provincia Domic.']
del data_reclamos['Distrito Domic.']

### Correccion de fechas
Se incorporo un archivo de centros poblados para facilitarnos la correcion

In [6]:
data_reclamos['fecha_fase1'] = pd.to_datetime(data_reclamos['Fase1_presen'], format='%d/%m/%Y')
data_reclamos['fecha_fase2'] = pd.to_datetime(data_reclamos['TEMPORAL'], format='%d/%m/%Y')
data_reclamos['fecha_fase3'] = pd.to_datetime(data_reclamos['CALIFFINAL'], format='%d/%m/%Y')
data_reclamos['fecha_fase4'] = pd.to_datetime(data_reclamos['Fase4_generac'], format='%d/%m/%Y')
data_reclamos['fecha_fase5'] = pd.to_datetime(data_reclamos['Fase5_revisac'], format='%d/%m/%Y')
data_reclamos['fecha_fase6'] = pd.to_datetime(data_reclamos['Fase6_enviac'], format='%d/%m/%Y')
data_reclamos['fecha_fase7'] = pd.to_datetime(data_reclamos['Fase7_firma_ACUSE'], format='%d/%m/%Y')

del data_reclamos['Fase1_presen']
del data_reclamos['TEMPORAL']
del data_reclamos['CALIFFINAL']
del data_reclamos['Fase4_generac']
del data_reclamos['Fase5_revisac']
del data_reclamos['Fase6_enviac']
del data_reclamos['Fase7_firma_ACUSE']

### Guardando archivo (PRTK)

In [7]:
data_reclamos['Cod Servicio'] = data_reclamos['Cod. Servicio'].fillna(-1).astype(int).astype(str)
data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].fillna(-1).astype(int).astype(str)

del data_reclamos['Cod. Servicio']

### Guardando archivo (PRTK)

In [8]:
file_name = 'data/pre_processed_data.csv'
data_reclamos.to_pickle(file_name)

In [9]:
data_reclamos.head()

,N° Reclamo,Nombre,N° Docum.,Servicio,Cod Motivo,Motivo,Cod. Atm,Det. AtmError,N° Cajero,Monto,...,DISTRITO,IS_LIMA,fecha_fase1,fecha_fase2,fecha_fase3,fecha_fase4,fecha_fase5,fecha_fase6,fecha_fase7,Cod Servicio
N°,,,,,,,,,,,,,,,,,,,,,
1,002718-7736,ADAN HOLGUIN SOLIS,42672067,CAJEROS AUTOMATICOS,280800,PRCA - NO DISPENSÓ EFECTIVO Y/O VOUCHER,5.0,(cajero multired) COD.82-ERROR EN EL ATM,NaN,NaN,...,LOS OLIVOS,True,2018-12-31,2019-01-17,2019-01-17,2019-01-21,2019-01-21,2019-01-17,2019-01-17,2200
2,067618-34,ADANAQUE TIZON MARIA LILIANA,45056104,CAJEROS AUTOMATICOS,280800,PRCA - NO DISPENSÓ EFECTIVO Y/O VOUCHER,5.0,(cajero multired) COD.82-ERROR EN EL ATM,774.0,200,...,TAMBOGRANDE,False,2018-12-31,2019-01-03,2019-01-03,2019-01-03,2019-01-03,2019-01-03,2019-01-03,2200
3,002718-7726,ADEMIR CHAVEZ SANCHEZ,41311452,CAJEROS AUTOMATICOS,280800,PRCA - NO DISPENSÓ EFECTIVO Y/O VOUCHER,99.0,(cajero multired) OTROS,NaN,20,...,VILLA MARIA DEL TRIUNFO,True,2018-12-31,2019-01-16,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2200
4,315118-10281,AGUERO OLIVAS JOHN RICHARD,40383052,CAJEROS AUTOMATICOS,280800,PRCA - NO DISPENSÓ EFECTIVO Y/O VOUCHER,99.0,(cajero multired) OTROS,1338.0,2580,...,HUANUCO,False,2018-12-31,2019-01-09,2019-01-09,2019-01-09,2019-01-09,2019-01-09,2019-01-09,2200
5,034618-9,AGUILAR VEGA JULIO EDUARDO,45451586,CAJEROS AUTOMATICOS,280800,PRCA - NO DISPENSÓ EFECTIVO Y/O VOUCHER,5.0,(cajero multired) COD.82-ERROR EN EL ATM,789.0,400,...,POMABAMBA,False,2018-12-31,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2200
